In [166]:
import pandas as pd
import numpy as np
from math import *
import itertools
from statistics import mode

In [63]:
data = pd.read_csv("./play_tennis.csv")

In [64]:
data = data.drop("day", axis=1)

In [65]:
data

,outlook,temp,humidity,wind,play
0,Sunny,100,High,Weak,No
1,Sunny,90,High,Strong,No
2,Overcast,80,High,Weak,Yes
3,Rain,70,High,Weak,Yes
4,Rain,60,Normal,Weak,Yes
5,Rain,50,Normal,Strong,No
6,Overcast,40,Normal,Strong,Yes
7,Sunny,70,High,Weak,No
8,Sunny,60,Normal,Weak,Yes
9,Rain,70,Normal,Weak,Yes


In [66]:
def catSplits(feature):
    combs = []
    for i in range(len(feature)+1):
        combs += [list(j) for j in itertools.combinations(feature, i)]
    
    combs.pop(0)
    combs.pop()

    for comb in combs:
        feat = feature.copy()
        if set(comb).issubset(set(feat)):
            for elem in comb:
                feat.remove(elem)
            if feat in combs:
                combs.remove(feat)
    
    return combs

def numSplits(feature):
    splits = []

    feature.sort()

    for i in range(len(feature) - 1):
        splits.append((feature[i] + feature[i+1])/2)

    return splits

def splitter(feature):
    if (feature.dtype == object):
        splits = catSplits(list(feature))
    elif (feature.dtype == np.int64):
        splits = numSplits(list(feature))
    else:
        splits = list(feature)

    return splits


In [276]:
class Node:
    def __init__(self, node_type, info, split, feat, left = None, right = None):
        self.type = node_type
        self.info = info
        self.left = left
        self.right = right
        self.parent = []
        self.split = split
        self.feat = feat

    def printNode(self, space):
        if self.type == 'root':
            return f"{space}Root Node\n{space}| Split Feature: {self.feat}\n{space}| Split Point: {self.split}\n{space}| Information Gain Of The Node: {round(self.info, 2)}"

        else:
            return f"{space}Intermediatte Node\n{space}| Split Feature: {self.feat}\n{space}| Split Point: {self.split}\n{space}| Information Gain Of The Node: {round(self.info, 2)}"


class leafNode:
    def __init__(self, info, label):
        self.info = info
        self.label = label

    def printNode(self, space):
        return f"{space}Leaf Node\n{space}| Information Gain Of The Node: {round(self.info, 2)}\n{space}| Predicted Label: {self.label}"

In [277]:
class DecisionTree:
    def __init__(self, data, purity_index = 0.95, target = "play"):
        self.data = data
        self.target = target
        self.index = purity_index
        self.root = []
        self.depth = 0

    def nodeSelector(self, data):
        cols = [col for col in data.columns if col != self.target]
        feats = [(data[col].unique(), col) for col in cols]
        splits_per_feats = [(splitter(feat), col) for (feat, col) in feats]
        infos = []

        for splits, feat in splits_per_feats:
            for split in splits:
                info, left, right = self.info_gainer(split, feat, data, data[self.target].unique(), self.target)
                infos.append((info, split, feat, left, right))

        infos.sort(key = lambda x: x[0], reverse=True)
        
        return infos[0]


    def treeMaker(self, node: Node, entro, split, feat, left, right):
        left_node_data = self.nodeSelector(left)
        right_node_data = self.nodeSelector(right)


        if ((left_node_data[0] > self.index) or (left_node_data[0] == 0) and ((right_node_data[0] > self.index) or (right_node_data[0] == 0))):
            label = self.to_terminal(left)
            left_node = leafNode(left_node_data[0], label)
            node.left = left_node

            label = self.to_terminal(right)
            right_node = leafNode(right_node_data[0], label)
            node.right = right_node
            return

        elif ((left_node_data[0] > self.index) or (left_node_data[0] == 0) and not ((right_node_data[0] > self.index) or (right_node_data[0] == 0))):
            label = self.to_terminal(left)
            left_node = leafNode(left_node_data[0], label)
            node.left = left_node

            right_node = Node("inter",right_node_data[0], right_node_data[1], right_node_data[2])
            node.right = right_node
            self.depth += 1
            self.treeMaker(right_node, *right_node_data)
            return

        elif (not (left_node_data[0] > self.index) or (left_node_data[0] == 0) and ((right_node_data[0] > self.index) or (right_node_data[0] == 0))):
            left_node = Node("inter", left_node_data[0], left_node_data[1], left_node_data[2])
            node.left = left_node
            self.depth += 1
            self.treeMaker(left_node, *left_node_data)

            label = self.to_terminal(right)
            right_node = leafNode(right_node_data[0], label)
            node.right = right_node
            return

        else:
            left_node = Node("inter", left_node_data[0], left_node_data[1], left_node_data[2])
            node.left = left_node
            self.depth += 1
            self.treeMaker(left_node, *left_node_data)

            right_node = Node("inter",right_node_data[0], right_node_data[1], right_node_data[2])
            node.right = right_node
            self.depth += 1
            self.treeMaker(right_node, *right_node_data)
            return


    def treeing(self):
        root_data = self.nodeSelector(self.data)
        root = Node("root", root_data[0], root_data[1], root_data[2])
        self.root = root

        self.treeMaker(root, *root_data)

    def to_terminal(self, data):
        outcomes = data[self.target]
        return mode(list(outcomes))


    def info_gainer(self, split, feat, data, classes, target):
        if type(split) == np.float64:
            left_data = data[data[feat] < split]
            right_data = data[data[feat] > split]
        else:
            left_data = data[data[feat].isin(split)]
            right_data = data[~data[feat].isin(split)]
            
        left_pro = len(left_data) / len(data)
        right_pro = len(right_data) / len(data)

        left_entro = self.entropier(left_data, data[target].unique(), target)
        right_entro = self.entropier(right_data, data[target].unique(), target)

        split_entro = (left_pro * left_entro) +  (right_pro * right_entro) 

        info_gain = self.entropier(data, classes, target) - split_entro

        return info_gain, left_data, right_data


    def entropier(self, data, classes, target):
        entro = 0
        for feat_class in classes:
            if len(data) == 0:
                class_pro = 0
            else:
                class_pro = len(data[data[target] == feat_class])/len(data)
            if class_pro == 0:
                class_entro = 0
            else:
                class_entro = -(class_pro * log2(class_pro))
            entro += class_entro
        return entro


    def print_tree(self, node, depth = 0):
        if isinstance(node, Node):
            print(node.printNode(int(depth * 4 ** 1.5)*'-'))
            self.print_tree(node.left, depth + 1)
            self.print_tree(node.right, depth + 1)
        else:
            print(node.printNode(int(depth * 4 ** 1.5)*'-'))
  
    def __repr__(self):
        if self.root == []:
            return "Tree Not Made Yet!"
        self.print_tree(self.root)
        return ""


In [278]:
tree = DecisionTree(data)

In [279]:
tree.treeing()

In [280]:
tree

Root Node
| Split Feature: outlook
| Split Point: ['Overcast']
| Information Gain Of The Node: 0.23
--------Leaf Node
--------| Information Gain Of The Node: 0.0
--------| Predicted Label: Yes
--------Intermediatte Node
--------| Split Feature: humidity
--------| Split Point: ['High']
--------| Information Gain Of The Node: 0.28
----------------Intermediatte Node
----------------| Split Feature: outlook
----------------| Split Point: ['Sunny']
----------------| Information Gain Of The Node: 0.32
------------------------Leaf Node
------------------------| Information Gain Of The Node: 0.0
------------------------| Predicted Label: No
------------------------Leaf Node
------------------------| Information Gain Of The Node: 1.0
------------------------| Predicted Label: Yes
----------------Leaf Node
----------------| Information Gain Of The Node: 0.72
----------------| Predicted Label: Yes
